In [ ]:
# Lab 1: 민원 자동 분류 + OpenAI API 실습
# 소방청 민원처리 AI Agent 교육

"""
학습 목표:
- OpenAI API 기본 사용법
- Temperature, Max Tokens, Streaming 이해
- 민원 자동 분류 시스템 구축
"""

# ============================================================
# 환경 설정
# ============================================================

import sys,os
if 'google.colab' in sys.modules:
    !pip install openai


from openai import OpenAI
import time
from typing import Dict

# API 키 설정
print("[API 키 입력] OpenAI API 키를 입력하세요")
api_key = " ".strip()
client = OpenAI(api_key=api_key)
print("환경 설정 완료!\n")

# ============================================================
# 실습 데이터
# ============================================================

sample_complaints = [
    "아파트 복도에 소화기가 없는데 괜찮은가요?",
    "식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?",
    "옆집에서 불이 났는데 어디로 신고해야 하나요?",
    "소방차 전용 주차구역에 차를 세웠다가 과태료 부과 통보를 받았습니다.",
]


[API 키 입력] OpenAI API 키를 입력하세요
환경 설정 완료!



In [2]:

# ============================================================
# 실습 1: 기본 API 호출
# ============================================================

print("="*60)
print("[실습 1] 기본 API 호출")
print("="*60)

def classify_complaint_basic(text: str) -> str:
    """
    민원을 카테고리별로 분류하는 기본 함수
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 분류 전문가입니다.
민원을 다음 카테고리로 분류하세요:
1. 화재신고/대응
2. 소방시설 점검/설치
3. 법규 위반/과태료
4. 일반 문의

카테고리와 간단한 이유를 반환하세요."""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=150
    )
    return response.choices[0].message.content

# 테스트
print("\n[테스트]")
test_complaint = sample_complaints[0]
print(f"민원: {test_complaint}")
print(f"\n분류 결과:")
print(classify_complaint_basic(test_complaint))


[실습 1] 기본 API 호출

[테스트]
민원: 아파트 복도에 소화기가 없는데 괜찮은가요?

분류 결과:
카테고리: 2. 소방시설 점검/설치  
이유: 아파트 복도의 소화기 설치 여부에 대한 문의로, 소방시설의 점검 및 설치와 관련된 사항입니다.


In [3]:

# ============================================================
# 실습 2: Temperature 비교
# ============================================================

print("\n\n" + "="*60)
print("[실습 2] Temperature 비교 - 창의성 조절")
print("="*60)

def compare_temperature(text: str):
    """
    동일 질문에 대해 온도를 다르게 설정하여 응답 비교
    - Temperature 0.0: 결정적, 일관된 답변
    - Temperature 1.0: 창의적, 다양한 답변
    """
    temperatures = [0.0, 0.5, 1.0]

    for temp in temperatures:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "소방 민원을 분류하세요."},
                {"role": "user", "content": text}
            ],
            temperature=temp,
            max_tokens=100
        )
        print(f"\n[Temperature = {temp}]")
        print(f"응답: {response.choices[0].message.content}")
        print(f"토큰 사용: {response.usage.total_tokens}")

# 실행
print("\n[테스트] 같은 질문을 3번 실행해봅니다.")
compare_temperature("소화기 사용법을 알려주세요")




[실습 2] Temperature 비교 - 창의성 조절

[테스트] 같은 질문을 3번 실행해봅니다.

[Temperature = 0.0]
응답: 소화기 사용법은 다음과 같습니다:

1. **소화기 확인**: 소화기가 정상적으로 작동하는지 확인합니다. 압력 게이지가 적정 범위에 있는지 확인하세요.

2. **안전핀 제거**: 소화기의 안전핀을 뽑아 안전장치를 해제합니다. 이때 핀을 잡고 있는 손가락에 주의하세요.

3. **노즐 방향 조정**: 소화
토큰 사용: 127

[Temperature = 0.5]
응답: 소화기 사용법은 다음과 같습니다:

1. **소화기 확인**: 소화기가 정상적으로 작동하는지 확인합니다. 압력 게이지가 적정 범위에 있는지, 소화기 본체에 손상이 없는지 점검합니다.

2. **안전핀 제거**: 소화기의 안전핀을 잡아당겨 제거합니다. 이 핀은 소화기가 우발적으로 작동하지 않도록 하는 역할을 합니다.


토큰 사용: 127

[Temperature = 1.0]
응답: 소화기 사용법은 다음과 같이 간단한 단계로 설명할 수 있습니다:

1. **안전 확인**: 화재가 발생한 경우, 먼저 안전한 거리를 유지하고, 주변 사람들에게 경고하세요.

2. **소화기 위치 확인**: 소화기가 가까운 곳에 있는지 확인합니다.

3. **소화기 꺼내기**: 소화기를 수직으로 들고, 안전 핀(잠금 장치
토큰 사용: 127


In [4]:

# ============================================================
# 실습 3: Max Tokens 비교
# ============================================================

print("\n\n" + "="*60)
print("[실습 3] Max Tokens 비교 - 응답 길이 제어")
print("="*60)

def compare_max_tokens(text: str):
    """
    토큰 제한에 따른 응답 차이 확인
    - 토큰이 적으면 답변이 짧고 핵심만
    - 토큰이 많으면 답변이 상세함
    """
    token_limits = [50, 150, 500]

    for max_tok in token_limits:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "소방 민원을 상세히 분류하고 처리 방안을 제시하세요."},
                {"role": "user", "content": text}
            ],
            temperature=0.5,
            max_tokens=max_tok
        )
        print(f"\n[Max Tokens = {max_tok}]")
        print(f"응답 길이: {len(response.choices[0].message.content)} 문자")
        print(f"응답:\n{response.choices[0].message.content}")
        print("-" * 50)

# 실행
print("\n[테스트] 상세한 질문으로 토큰 차이 확인")
compare_max_tokens("아파트 복도 소화기 설치 기준과 위반 시 처리 방법을 알려주세요")





[실습 3] Max Tokens 비교 - 응답 길이 제어

[테스트] 상세한 질문으로 토큰 차이 확인

[Max Tokens = 50]
응답 길이: 99 문자
응답:
아파트 복도에 소화기 설치 기준 및 위반 시 처리 방법에 대해 다음과 같이 상세히 설명하겠습니다.

### 아파트 복도 소화기 설치 기준

1. **설치 위치**:
   - 소화
--------------------------------------------------

[Max Tokens = 150]
응답 길이: 280 문자
응답:
아파트 복도 소화기 설치 기준과 위반 시 처리 방법에 대해 상세히 설명하겠습니다.

### 아파트 복도 소화기 설치 기준

1. **설치 위치**:
   - 소화기는 아파트 복도의 출입구, 계단, 엘리베이터 근처와 같은 주요 경로에 설치해야 합니다.
   - 각 층마다 적어도 1대 이상의 소화기를 설치해야 하며, 거리가 30미터 이내에 위치해야 합니다.

2. **소화기 종류**:
   - 일반적으로 사용되는 소화기는 ABC형 소화기로, 화재의 종류에 따라 다양한 물질에 효과적입니다.
   - 소
--------------------------------------------------

[Max Tokens = 500]
응답 길이: 919 문자
응답:
아파트 복도에 소화기 설치 기준과 위반 시 처리 방법에 대해 아래와 같이 상세히 설명하겠습니다.

### 아파트 복도 소화기 설치 기준

1. **설치 위치**:
   - 아파트 복도 내의 소화기는 입구와 각 층의 복도에 설치해야 하며, 각 층마다 최소 1개 이상 설치해야 합니다.
   - 소화기는 출입구에서 30m 이내의 거리에 위치해야 하며, 시각적으로 쉽게 접근할 수 있는 곳에 설치해야 합니다.

2. **소화기 종류**:
   - 일반적으로 사용되는 소화기는 분말 소화기, 이산화탄소 소화기 등이며, 아파트의 경우 주로 ABC 소화기가 권장됩니다.
   - 소화기의 용량은 최소 2kg 이상이어야 하며, 대형

In [5]:
# ============================================================
# 실습 4: 스트리밍 vs 일반 응답
# ============================================================

print("\n\n" + "="*60)
print("[실습 4] 스트리밍 vs 일반 응답 속도 비교")
print("="*60)

# 일반 응답
print("\n[일반 응답 방식]")
start = time.time()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "소방시설 점검 주기를 설명하세요"}],
    stream=False
)
normal_time = time.time() - start
print(f"소요 시간: {normal_time:.2f}초")
print(f"응답: {response.choices[0].message.content}\n")

# 스트리밍 응답
print("\n[스트리밍 응답 방식]")
print("(실시간으로 단어가 출력됩니다)")
print("-" * 50)
start = time.time()
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "소방시설 점검 주기를 설명하세요"}],
    stream=True
)
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
stream_time = time.time() - start
print(f"\n{'-' * 50}")
print(f"소요 시간: {stream_time:.2f}초")
print(f"\n참고: 총 시간은 비슷하지만, 스트리밍은 첫 단어가 빨리 나와서 체감 속도가 빠릅니다!")





[실습 4] 스트리밍 vs 일반 응답 속도 비교

[일반 응답 방식]
소요 시간: 5.39초
응답: 소방시설 점검 주기는 소방시설의 안전성과 기능을 유지하기 위해 정기적으로 점검을 실시하는 주기를 의미합니다. 각종 소방시설에는 점검 주기가 법적 요구사항에 따라 다르며, 일반적으로 다음과 같은 기준이 적용됩니다.

1. **소화기**: 매월 1회 점검이 권장되며, 1년에 1번 전문적인 검사를 받는 것이 좋습니다.

2. **스프링클러 시스템**: 연 1회 정기 점검이 필요하며, 5년마다 더 상세한 검사가 권장됩니다.

3. **경보기 및 감지기**: 연 1회 점검이 필요하며, 배터리 교체 및 기능 점검도 포함됩니다.

4. **소화전**: 연 1회 점검이 필요하며, 사용 상태 확인과 물 공급 상태 점검이 포함됩니다.

5. **소화펌프**: 연 1회 점검과 함께 기능 테스트를 수행해야 하며, 3-5년마다 실질적인 성능 검사가 필요합니다.

이 외에도 각 시설의 종류와 설치 환경에 따라 점검 주기가 달라질 수 있습니다. 따라서 소방시설 점검은 관련 법규 및 지침에 따라 정확하게 이행되어야 하며, 이를 통해 화재로 인한 피해를 최소화할 수 있습니다.


[스트리밍 응답 방식]
(실시간으로 단어가 출력됩니다)
--------------------------------------------------
소방시설 점검 주기는 소방시설이 정상적으로 작동할 수 있도록 정기적으로 점검하는 주기를 말합니다. 이는 화재 발생 시 인명과 재산을 보호하기 위한 필수적인 절차로, 소방법 및 관련 규정에 따라 정해진 주기가 있습니다.

일반적으로 소방시설의 점검 주기는 다음과 같습니다:

1. **소화기**: 연 1회 이상 점검.
2. **소화전**: 연 1회 이상 점검.
3. **화재감지기 및 경보시설**: 연 1회 이상 점검.
4. **스프링클러 시스템**: 연 1회 이상 점검.
5. **배연설비**: 연 1회 이상 점검.
6. **비상조명 및 유도등**: 연 1회 이상 점검.



In [6]:
# ============================================================
# 실습 5: 최종 분류 함수 (구조화된 출력)
# ============================================================

print("\n\n" + "="*60)
print("[실습 5] 최종 민원 분류 시스템")
print("="*60)

def classify_complaint(text: str) -> Dict[str, str]:
    """
    민원 분류 - 구조화된 결과 반환
    이 함수는 Lab 2-5에서 재사용됩니다.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )

    result_text = response.choices[0].message.content

    return {
        "원문": text,
        "분류_결과": result_text,
        "전체_응답": result_text
    }

# 전체 샘플 테스트
print("\n[모든 민원 자동 분류]")
print("="*60)

for i, complaint in enumerate(sample_complaints, 1):
    print(f"\n{i}. 민원: {complaint}")
    result = classify_complaint(complaint)
    print(f"\n분류 결과:")
    print(result['분류_결과'])
    print("-"*60)





[실습 5] 최종 민원 분류 시스템

[모든 민원 자동 분류]

1. 민원: 아파트 복도에 소화기가 없는데 괜찮은가요?

분류 결과:
카테고리: [소방시설]  
긴급도: [보통]  
키워드: [소화기, 아파트, 안전]  
요약: 아파트 복도에 소화기가 없는 것은 안전에 문제가 될 수 있습니다.
------------------------------------------------------------

2. 민원: 식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?

분류 결과:
카테고리: 일반문의  
긴급도: 보통  
키워드: [소방시설, 점검, 식당]  
요약: 식당의 소방시설 점검 주기에 대한 문의입니다.
------------------------------------------------------------

3. 민원: 옆집에서 불이 났는데 어디로 신고해야 하나요?

분류 결과:
카테고리: [화재신고]  
긴급도: [높음]  
키워드: [화재, 신고, 긴급]  
요약: 옆집 화재 발생 시 즉시 119에 신고해야 합니다.
------------------------------------------------------------

4. 민원: 소방차 전용 주차구역에 차를 세웠다가 과태료 부과 통보를 받았습니다.

분류 결과:
카테고리: 법규위반  
긴급도: 보통  
키워드: 소방차, 주차구역, 과태료  
요약: 소방차 전용 주차구역에 주차하여 과태료를 부과받은 사례입니다.
------------------------------------------------------------


In [8]:
# ============================================================
# Lab 1 완료
# ============================================================

print("\n" + "="*60)
print("[완료] Lab 1 완료!")
print("="*60)
print("""
학습 내용:
1. OpenAI API 기본 호출 방법
2. Temperature로 창의성 조절
3. Max Tokens로 응답 길이 제어
4. Streaming으로 실시간 응답
5. 구조화된 민원 분류 시스템
classify_complaint() 함수를 기억!
""")


[완료] Lab 1 완료!

학습 내용:
1. OpenAI API 기본 호출 방법
2. Temperature로 창의성 조절
3. Max Tokens로 응답 길이 제어
4. Streaming으로 실시간 응답
5. 구조화된 민원 분류 시스템
classify_complaint() 함수를 기억!

